In [1]:
import pandas as pd
import numpy as np
import os
from PIL import Image

In [2]:
import random
from dataclasses import dataclass
from typing import Any, List, Dict, Optional, Union

import cv2
import torch
import numpy as np
from PIL import Image
import plotly.express as px
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from transformers import  pipeline   

/home/syasna_smith_edu/.conda/envs/mothitor/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
df = pd.read_csv("/work/pi_mrobson_smith_edu/mothitor/code_main/efficientnet_data/reshape_pass_5/detections.csv")

In [ ]:
#read the first 100 unique image_name rows
first_100 = df['image_name'].unique()[:100]
# Create a new DataFrame with the first 100 unique image_name rows
df_first_100 = df[df['image_name'].isin(first_100)]

In [17]:
#take the detection results -> return annoated image 
def annotate(image, annotations_df) -> np.ndarray:
    # Convert PIL Image to OpenCV format
    image_cv2 = Image.open(image).convert("RGB")
    image_cv2 = np.array(image_cv2)
   # image_cv2 = np.array(image) if isinstance(image, Image.Image) else image
   # image_cv2 = cv2.cvtColor(image_cv2, cv2.COLOR_RGB2BGR)

    # Iterate over detections and add bounding boxes from annoations_df
    for index, row in annotations_df.iterrows():
        label = row['label']
        score = row['detection_score']
        xmin = row['xmin']
        ymin = row['ymin']
        xmax = row['xmax']
        ymax = row['ymax']
        # Sample a random color for each detection
        color = np.random.randint(0, 256, size=3)

        # Draw bounding box
        cv2.rectangle(image_cv2, (xmin, ymin), (xmax, ymax), color.tolist(), 2)
        cv2.putText(image_cv2, f'{label}: {score:.2f}', (xmin, ymin - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color.tolist(), 2)
    return cv2.cvtColor(image_cv2, cv2.COLOR_BGR2RGB)

In [ ]:
for index, row in df.iterrows():
    image_path = row['image_name']
    image_path = os.path.join("/work/pi_mrobson_smith_edu/mothitor/data/Mothitor4.0Pics", image_path)
    df_sub = df[df['image_name'] == row['image_name']]
    image = annotate(image_path, df_sub)